In [18]:
# pip install sqlalchemy psycopg2-binary pandas

In [19]:
import pandas as pd
import numpy as np
import requests
import sqlite3
import os
import time
from google.colab import drive

# **Setup**

In [20]:
drive.mount("/content/drive")

BASE_DIR = "/content/drive/MyDrive/dataset/bigdata/uas"
RAW_DIR = f"{BASE_DIR}/raw"
DB_PATH = f"{BASE_DIR}/warehouse.db"

os.makedirs(RAW_DIR, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **EXTRACT**

In [21]:
# =========================================================
# EXTRACT
# =========================================================
def extract_etl_source1():
    start = time.time()

    path = f"{BASE_DIR}/credit_card_transactions.csv"
    df = pd.read_csv(path)

    raw_path = f"{RAW_DIR}/raw_credit_card.csv"
    df.to_csv(raw_path, index=False)

    print("[EXTRACT] Credit Card Dataset")
    print("Rows:", df.shape[0], "Cols:", df.shape[1])
    print("Execution time:", round(time.time() - start, 2), "sec\n")

    return df


def extract_etl_source2():
    start = time.time()

    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": "FEDFUNDS",
        "api_key": "51c9062a781c9b5719b643681e26d34b",
        "file_type": "json",
        "frequency": "m"
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()

    data = resp.json()["observations"]
    df = pd.DataFrame(data)

    raw_path = f"{RAW_DIR}/raw_fred_fedfunds.csv"
    df.to_csv(raw_path, index=False)

    print("[EXTRACT] FRED FEDFUNDS")
    print("Rows:", df.shape[0], "Cols:", df.shape[1])
    print("Execution time:", round(time.time() - start, 2), "sec\n")

    return df

# **TRANSFORM**

In [22]:
df = extract_etl_source1()
df_fred = extract_etl_source2()

[EXTRACT] Credit Card Dataset
Rows: 1296675 Cols: 24
Execution time: 38.37 sec

[EXTRACT] FRED FEDFUNDS
Rows: 858 Cols: 4
Execution time: 0.26 sec



In [23]:
print("===== TRANSFORM START =====")
print("Initial shape:", df.shape, "\n")

===== TRANSFORM START =====
Initial shape: (1296675, 24) 



In [24]:
# ---------------------------------------------------------
# 1. STANDARDISASI NAMA KOLOM
# ---------------------------------------------------------
print("[CLEANING] Standardisasi Nama Kolom")

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(r"[^\w]+", "_", regex=True)  # FIX: hapus ':' dll
)

# FIX: drop kolom index bawaan CSV
if "unnamed_0" in df.columns:
    df = df.drop(columns=["unnamed_0"])

print("Columns after standardization:")
print(df.columns.tolist())
print("-" * 60, "\n")

[CLEANING] Standardisasi Nama Kolom
Columns after standardization:
['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'merch_zipcode']
------------------------------------------------------------ 



In [25]:
# ---------------------------------------------------------
# 2. DUPLICATE REMOVAL
# ---------------------------------------------------------
print("[CLEANING] Drop Duplicate (trans_num)")
rows_before = len(df)

df = df.drop_duplicates(subset=["trans_num"])

print("Rows before:", rows_before)
print("Rows after :", len(df))
print("-" * 60, "\n")

[CLEANING] Drop Duplicate (trans_num)
Rows before: 1296675
Rows after : 1296675
------------------------------------------------------------ 



In [26]:
# ---------------------------------------------------------
# 3. DATETIME STANDARDIZATION
# ---------------------------------------------------------
print("[CLEANING] Parsing Datetime Columns")

df["trans_date_trans_time"] = pd.to_datetime(df["trans_date_trans_time"], errors="coerce")
df["dob"] = pd.to_datetime(df["dob"], errors="coerce")

print(df[["trans_date_trans_time", "dob"]].dtypes)
print("-" * 60, "\n")

[CLEANING] Parsing Datetime Columns
trans_date_trans_time    datetime64[ns]
dob                      datetime64[ns]
dtype: object
------------------------------------------------------------ 



In [27]:
# ---------------------------------------------------------
# 4. MISSING VALUE HANDLING
# ---------------------------------------------------------
print("[CLEANING] Missing Value Handling")

critical_cols = ["cc_num", "amt", "category", "gender"]
print("Missing before:")
print(df[critical_cols].isna().sum())

# FIX: hanya drop kolom krusial
df = df.dropna(subset=critical_cols)

# FIX: kolom non-kritis diisi default
df["merch_zipcode"] = df["merch_zipcode"].fillna(-1)

print("Rows after cleaning:", len(df))
print("-" * 60, "\n")

[CLEANING] Missing Value Handling
Missing before:
cc_num      0
amt         0
category    0
gender      0
dtype: int64
Rows after cleaning: 1296675
------------------------------------------------------------ 



In [28]:
# ---------------------------------------------------------
# 5. OUTLIER HANDLING (IQR - CAPPING)
# ---------------------------------------------------------
print("[CLEANING] Outlier Handling (IQR - Capping)")

Q1 = df["amt"].quantile(0.25)
Q3 = df["amt"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

# FIX: gunakan capping agar sinyal fraud tidak hilang
df["amt"] = df["amt"].clip(lower=lower, upper=upper)

print("Lower bound:", round(lower, 2))
print("Upper bound:", round(upper, 2))
print("-" * 60, "\n")

[CLEANING] Outlier Handling (IQR - Capping)
Lower bound: -100.58
Upper bound: 193.38
------------------------------------------------------------ 



In [29]:
# ---------------------------------------------------------
# 6. FEATURE ENGINEERING
# ---------------------------------------------------------
print("[FEATURE ENGINEERING]")

df["year"] = df["trans_date_trans_time"].dt.year
df["month"] = df["trans_date_trans_time"].dt.month
df["hour"] = df["trans_date_trans_time"].dt.hour
df["year_month"] = df["trans_date_trans_time"].dt.to_period("M").astype(str)

df["age"] = (df["trans_date_trans_time"] - df["dob"]).dt.days // 365

# FIX: buang age negatif
df = df[df["age"] >= 0]

df["amt_log"] = np.log1p(df["amt"])

print("Total columns:", df.shape[1])
print("-" * 60, "\n")

[FEATURE ENGINEERING]
Total columns: 29
------------------------------------------------------------ 



In [30]:
# ---------------------------------------------------------
# 7. ENCODING
# ---------------------------------------------------------
print("[ENCODING] Categorical Encoding")

# FIX: aman jika ada nilai selain M/F
df["gender_encoded"] = df["gender"].map({"M": 0, "F": 1}).fillna(-1)

df = pd.get_dummies(df, columns=["category"], prefix="cat")

print("Columns after encoding:", df.shape[1])
print("-" * 60, "\n")

[ENCODING] Categorical Encoding
Columns after encoding: 43
------------------------------------------------------------ 



In [31]:
# ---------------------------------------------------------
# 8. NORMALIZATION
# ---------------------------------------------------------
print("[NORMALIZATION] Min-Max Scaling")

df["amt_norm"] = (
    (df["amt"] - df["amt"].min()) /
    (df["amt"].max() - df["amt"].min() + 1e-9)  # FIX: avoid div by zero
)

df["age_norm"] = (
    (df["age"] - df["age"].min()) /
    (df["age"].max() - df["age"].min() + 1e-9)
)

print(df[["amt_norm", "age_norm"]].head())
print("-" * 60, "\n")

[NORMALIZATION] Min-Max Scaling
   amt_norm  age_norm
0  0.020637  0.207317
1  0.552203  0.329268
2  1.000000  0.524390
3  0.228720  0.475610
4  0.212917  0.231707
------------------------------------------------------------ 



In [32]:
# ---------------------------------------------------------
# 9. ENRICHMENT (FRED)
# ---------------------------------------------------------
print("[ENRICHMENT] Join FRED Interest Rate")

df_fred["date"] = pd.to_datetime(df_fred["date"], errors="coerce")
df_fred["year_month"] = df_fred["date"].dt.to_period("M").astype(str)
df_fred["interest_rate"] = pd.to_numeric(df_fred["value"], errors="coerce")

df_fred = df_fred[["year_month", "interest_rate"]].dropna()

df_final = df.merge(df_fred, on="year_month", how="left")

print(df_final[["year_month", "amt", "interest_rate"]].head())
print("===== TRANSFORM END =====\n")

[ENRICHMENT] Join FRED Interest Rate
  year_month      amt  interest_rate
0    2019-01    4.970            2.4
1    2019-01  107.230            2.4
2    2019-01  193.375            2.4
3    2019-01   45.000            2.4
4    2019-01   41.960            2.4
===== TRANSFORM END =====



In [33]:
# =========================================================
# DATA QUALITY VALIDATION
# =========================================================
assert df_final["trans_num"].is_unique
assert df_final["amt"].notna().all()
assert (df_final["amt"] >= 0).all()
assert np.issubdtype(df_final["interest_rate"].dtype, np.number)
assert df_final["year"].between(2000, 2030).all()
assert df_final["cc_num"].notna().all()

print("✅ DATA QUALITY CHECK PASSED")

✅ DATA QUALITY CHECK PASSED


# **LOAD TO GOOGLE DRIVE (CSV)**

In [34]:
# =========================================================
# LOAD TO GOOGLE DRIVE (CSV)
# =========================================================
OUTPUT_DIR = f"{BASE_DIR}/processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

output_csv_path = f"{OUTPUT_DIR}/credit_card_transactions_cleaned.csv"

df_final.to_csv(output_csv_path, index=False)

print("✅ Data berhasil disimpan ke Google Drive")
print("Path:", output_csv_path)
print("Rows:", df_final.shape[0], "Cols:", df_final.shape[1])


✅ Data berhasil disimpan ke Google Drive
Path: /content/drive/MyDrive/dataset/bigdata/uas/processed/credit_card_transactions_cleaned.csv
Rows: 1296675 Cols: 46
